In [1]:
import pandas as pd
import numpy as np
from scipy.stats import kendalltau

In [2]:
test_te=pd.read_csv("test_te.csv")
test_tr=pd.read_csv("test_tr.csv")

In [4]:
rec_index=pd.read_csv("rec_indexes.csv")
target=pd.read_csv("targets.csv")
rec_index =rec_index.drop(columns=["Unnamed: 0"])
target =target.drop(columns=["Unnamed: 0"])

## user_id对齐

In [4]:
unique_user_id = test_te["user_id"].unique()

In [5]:
test_user = pd.DataFrame(0, index=np.arange(606), columns=['user_id'])
user_id = unique_user_id
for i in range(0,606):
    tar = pd.DataFrame()
    tar = target.iloc[i]
    tar_item = set(tar[tar == 1].index.astype(int))
    for id in user_id:
        item_set = set(test_te.loc[test_te['user_id']==id]['item_id'])
        if tar_item == item_set:
            test_user.iloc[i] = id
            index_to_delete = np.where(user_id == id)[0]
            user_id = np.delete(user_id, index_to_delete)
            break
test_user

,user_id
0,2531
1,2704
2,2389
3,5877
4,5188
...,...
601,589
602,662
603,5493
604,3012


In [6]:
rec_list = pd.concat([test_user, rec_index], axis=1)
rec_list

,user_id,0,1,2,3,4,5,6,7,8,9
0,2531,920,902,1247,914,899,898,1947,915,527,1183
1,2704,1250,593,1207,912,923,1208,1952,1299,608,1247
2,2389,34,1,1136,1197,1214,1288,858,1079,260,1617
3,5877,2858,2762,2997,1210,260,3793,3578,2355,1270,2959
4,5188,1270,1721,1097,588,919,260,527,1210,318,3105
...,...,...,...,...,...,...,...,...,...,...,...
601,589,1,318,1265,1197,34,2028,2396,2716,527,260
602,662,1566,364,2628,1967,2005,480,260,1210,1196,1032
603,5493,2028,527,1198,1266,1265,1387,1834,2396,47,1394
604,3012,2916,3702,924,2288,1676,2968,3471,1206,1584,1127


In [7]:
ndcg = pd.DataFrame(columns=['dcg', 'idcg'])

In [8]:
user_ndcg = pd.concat([test_user, ndcg], axis=1)

In [9]:
user_ndcg

,user_id,dcg,idcg
0,2531,NaN,NaN
1,2704,NaN,NaN
2,2389,NaN,NaN
3,5877,NaN,NaN
4,5188,NaN,NaN
...,...,...,...
601,589,NaN,NaN
602,662,NaN,NaN
603,5493,NaN,NaN
604,3012,NaN,NaN


In [10]:
rank_discount = 1.0 / np.log2(np.arange(2, 10 + 2))

In [11]:
NDCG = 0
for i in range(606):
    dcg = 0
    idcg = 0
    for j in range(10):
        dcg += (2 ** target.iloc[i,rec_index.iloc[i,j]] - 1) / np.log2(j + 2)
    user_ndcg.iloc[i,1] = dcg
    idcg = (rank_discount[: min(int(sum(target.iloc[i])), 10)]).sum()
    user_ndcg.iloc[i,2] = idcg
    NDCG += dcg/idcg
NDCG = NDCG/606

In [12]:
user_ndcg

,user_id,dcg,idcg
0,2531,0.817529,2.561606
1,2704,2.464263,3.953465
2,2389,0.634363,4.543559
3,5877,0.63093,2.13093
4,5188,1.5,4.543559
...,...,...,...
601,589,0.5,2.561606
602,662,1.076393,4.543559
603,5493,3.886322,4.543559
604,3012,1.37399,4.543559


In [13]:
NDCG

0.3116091181550189

## 计算VAE的公平性指标

参数设置 Movielens 600 LastFM 750

In [854]:
n_considered = 600

In [505]:
n_considered = 750

路径设置

In [1233]:
path = "ML_GLM/7/0_1/"

## chi100 gender

In [1234]:
s1 = pd.read_csv(path+"s1_chi_gender.csv")
s0 = pd.read_csv(path+"s0_chi_gender.csv")
s1 = s1.drop(columns="Unnamed: 0")
s0 = s0.drop(columns="Unnamed: 0")
df1 = s1.iloc[:,0].to_numpy()
df0 = s0.iloc[:,0].to_numpy()
df = [df0,df1]

In [1235]:
df

[array([  0., 239.,  10., ...,   0.,   0.,  23.]),
 array([  0., 101.,   2., ...,   0.,   0.,   9.])]

In [1236]:
ct = np.concatenate((df[0].reshape(-1, 1), df[1].reshape(-1, 1)), axis=1)

In [1237]:
n_items = ct.shape[0]
adjusted_exps = np.zeros(ct.shape)

In [1238]:
scores_0 = pd.read_csv(path+"scores0_gender.csv")
scores_1 = pd.read_csv(path+"scores1_gender.csv")

In [1239]:
scores_0 = scores_0.drop(columns="Unnamed: 0")
scores_1 = scores_1.drop(columns="Unnamed: 0")

In [1240]:
scores0 = scores_0.to_numpy()
scores1 = scores_1.to_numpy()

In [1241]:
n0 = scores0.shape[0]
n1 = scores1.shape[0]

In [1242]:
for i in range(n_items):
    n_obs = ct[i].sum()

    # User who have interacted with an item cannot be recommended the same item
    # I.e., we subtract the number of times we have flagged the item with
    # a score lower than 0 for each user group
    n_available0 = n0 - np.count_nonzero(scores0[:, i] < 0)
    n_available1 = n1 - np.count_nonzero(scores1[:, i] < 0)
    n_available = n_available0 + n_available1

    adjusted_exps[i, 0] = (n_available0 / n_available) * n_obs
    adjusted_exps[i, 1] = (n_available1 / n_available) * n_obs

In [1243]:
index_partition = np.argpartition(-adjusted_exps.sum(1), n_considered)
adjusted_exps = adjusted_exps[index_partition[:n_considered]]
ct = ct[index_partition[:n_considered]]
chi2_ad = (ct - adjusted_exps) ** 2 / adjusted_exps

In [1244]:
chi2_ad.sum()

3278.0419447970517

## kendall100 gender

In [1245]:
s1 = pd.read_csv(path+"s1_ken_gender.csv")
s0 = pd.read_csv(path+"s0_ken_gender.csv")
s1 = s1.drop(columns="Unnamed: 0")
s0 = s0.drop(columns="Unnamed: 0")
df1 = s1.iloc[:,0].to_numpy()
df0 = s0.iloc[:,0].to_numpy()
df = [df0,df1]

In [1246]:
df

[array([ 0.        , 54.41903249,  1.90409866, ...,  0.        ,
         0.        ,  4.57954908]),
 array([ 0.        , 25.75628138,  0.37861113, ...,  0.        ,
         0.        ,  2.06008941])]

In [1247]:
def get_top_k_aggregated_ranks(all_ranks, agg_k):
    out_ranks = []
    for s_ranks in all_ranks:
        recommended_items = np.argpartition(-s_ranks, agg_k)[:agg_k]
        item_order = np.argsort(-s_ranks[recommended_items])
        out_ranks.append(recommended_items[item_order])
    return out_ranks

In [1248]:
ranks = get_top_k_aggregated_ranks(df, 100)

In [1249]:
def extended_tau(list_a, list_b):
    """Calculate the extended Kendall tau from two lists."""
    ranks = join_ranks(create_rank(list_a), create_rank(list_b)).fillna(len(list_a))
    dummy_df = pd.DataFrame(
        [{"rank_a": len(list_a), "rank_b": len(list_b)} for i in range(len(list_a) * 2 - len(ranks))]
    )
    total_df = ranks.append(dummy_df)
    return scale_tau(len(list_a), kendalltau(total_df["rank_a"], total_df["rank_b"])[0])


def scale_tau(length, value):
    """Scale an extended tau correlation such that it falls in [-1, +1]."""
    n_0 = 2 * length * (2 * length - 1)
    n_a = length * (length - 1)
    n_d = n_0 - n_a
    min_tau = (2.0 * n_a - n_0) / (n_d)
    return 2 * (value - min_tau) / (1 - min_tau) - 1

def create_rank(a):
    """Convert an ordered list to a DataFrame with ranks."""
    return pd.DataFrame(zip(a, range(len(a))), columns=["key", "rank"]).set_index("key")


def join_ranks(rank_a, rank_b):
    """Join two rank DataFrames."""
    return rank_a.join(rank_b, lsuffix="_a", rsuffix="_b", how="outer")

In [1250]:
extended_tau(ranks[0], ranks[1])

C:\Users\shaoyang\AppData\Local\Temp\ipykernel_17752\3504980276.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = ranks.append(dummy_df)


0.5395591182364727

## chi100 age

In [1251]:
s1 = pd.read_csv(path+"s1_chi_age.csv")
s0 = pd.read_csv(path+"s0_chi_age.csv")
s1 = s1.drop(columns="Unnamed: 0")
s0 = s0.drop(columns="Unnamed: 0")
df1 = s1.iloc[:,0].to_numpy()
df0 = s0.iloc[:,0].to_numpy()
df = [df0,df1]

In [1252]:
df

[array([  0., 279.,  14., ...,   0.,   0.,  21.]),
 array([ 0., 75.,  3., ...,  0.,  0., 10.])]

In [1253]:
ct = np.concatenate((df[0].reshape(-1, 1), df[1].reshape(-1, 1)), axis=1)
n_items = ct.shape[0]
adjusted_exps = np.zeros(ct.shape)
scores_0 = pd.read_csv(path+"scores0_age.csv")
scores_1 = pd.read_csv(path+"scores1_age.csv")
scores_0 = scores_0.drop(columns="Unnamed: 0")
scores_1 = scores_1.drop(columns="Unnamed: 0")
scores0 = scores_0.to_numpy()
scores1 = scores_1.to_numpy()
n0 = scores0.shape[0]
n1 = scores1.shape[0]

In [1254]:
for i in range(n_items):
    n_obs = ct[i].sum()

    # User who have interacted with an item cannot be recommended the same item
    # I.e., we subtract the number of times we have flagged the item with
    # a score lower than 0 for each user group
    n_available0 = n0 - np.count_nonzero(scores0[:, i] < 0)
    n_available1 = n1 - np.count_nonzero(scores1[:, i] < 0)
    n_available = n_available0 + n_available1

    adjusted_exps[i, 0] = (n_available0 / n_available) * n_obs
    adjusted_exps[i, 1] = (n_available1 / n_available) * n_obs

In [1255]:
index_partition = np.argpartition(-adjusted_exps.sum(1), n_considered)
adjusted_exps = adjusted_exps[index_partition[:n_considered]]
ct = ct[index_partition[:n_considered]]
chi2_ad = (ct - adjusted_exps) ** 2 / adjusted_exps
chi2_ad.sum()

2282.9265240109494

## ken100 age

In [1256]:
s1 = pd.read_csv(path+"s1_ken_age.csv")
s0 = pd.read_csv(path+"s0_ken_age.csv")
s1 = s1.drop(columns="Unnamed: 0")
s0 = s0.drop(columns="Unnamed: 0")
df1 = s1.iloc[:,0].to_numpy()
df0 = s0.iloc[:,0].to_numpy()
df = [df0,df1]

In [1257]:
df

[array([ 0.        , 67.83995825,  2.43192052, ...,  0.        ,
         0.        ,  3.69848169]),
 array([ 0.        , 15.87079142,  0.71815271, ...,  0.        ,
         0.        ,  2.34802342])]

In [1258]:
def get_top_k_aggregated_ranks(all_ranks, agg_k):
    out_ranks = []
    for s_ranks in all_ranks:
        recommended_items = np.argpartition(-s_ranks, agg_k)[:agg_k]
        item_order = np.argsort(-s_ranks[recommended_items])
        out_ranks.append(recommended_items[item_order])
    return out_ranks

In [1259]:
ranks = get_top_k_aggregated_ranks(df, 100)

In [1260]:
def extended_tau(list_a, list_b):
    """Calculate the extended Kendall tau from two lists."""
    ranks = join_ranks(create_rank(list_a), create_rank(list_b)).fillna(len(list_a))
    dummy_df = pd.DataFrame(
        [{"rank_a": len(list_a), "rank_b": len(list_b)} for i in range(len(list_a) * 2 - len(ranks))]
    )
    total_df = ranks.append(dummy_df)
    return scale_tau(len(list_a), kendalltau(total_df["rank_a"], total_df["rank_b"])[0])


def scale_tau(length, value):
    """Scale an extended tau correlation such that it falls in [-1, +1]."""
    n_0 = 2 * length * (2 * length - 1)
    n_a = length * (length - 1)
    n_d = n_0 - n_a
    min_tau = (2.0 * n_a - n_0) / (n_d)
    return 2 * (value - min_tau) / (1 - min_tau) - 1

def create_rank(a):
    """Convert an ordered list to a DataFrame with ranks."""
    return pd.DataFrame(zip(a, range(len(a))), columns=["key", "rank"]).set_index("key")


def join_ranks(rank_a, rank_b):
    """Join two rank DataFrames."""
    return rank_a.join(rank_b, lsuffix="_a", rsuffix="_b", how="outer")

In [1261]:
extended_tau(ranks[0], ranks[1])

C:\Users\shaoyang\AppData\Local\Temp\ipykernel_17752\3504980276.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = ranks.append(dummy_df)


0.5936673346693389